In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Problem 1:   
a) What is 30 [dB] in real number scale?  
b) What is 6 [dB] in real number scale? 

In [ ]:
# ANSWER
problem_a = 10**(0.1*30)
problem_b = 10**(0.1*6)

print(problem_a)
print(problem_b)

# a = 1000
# b = 4

## QPSK modulation 

Data : binary data. For example, [0 1 1 0 0 1 0 ...]  
QPSK modulation:   
$00 \rightarrow S_1: \left(\sqrt{\mathcal{E}/2},\sqrt{\mathcal{E}/2}\right)$,   
$01 \rightarrow S_2: \left(\sqrt{\mathcal{E}/2},-\sqrt{\mathcal{E}/2}\right)$,   
$11 \rightarrow S_3: \left(-\sqrt{\mathcal{E}/2},-\sqrt{\mathcal{E}/2}\right)$,   
$10 \rightarrow S_4: \left(-\sqrt{\mathcal{E}/2},\sqrt{\mathcal{E}/2}\right)$. 


$x$ : QPSK modulated signal. For example, $[S_2, S_4, S_2, \cdots]$  
$y = x + n$ : received signal  
$n = \left(n_1, n_2 \right)$ : AWGN noise, $n_1, n_2 \sim \mathcal{N}(0,\sigma^2/2)$

Ex) What is the variance of $n$?


In [ ]:
N_symbols = 20
E = 1
sigma = 1

data = np.random.randint(0, 2, N_symbols)
print('binary data:',data)

S_1 = np.array([np.sqrt(E/2)+np.sqrt(E/2)*1j])
S_2 = np.array([np.sqrt(E/2)-np.sqrt(E/2)*1j])
S_3 = np.array([-np.sqrt(E/2)-np.sqrt(E/2)*1j])
S_4 = np.array([-np.sqrt(E/2)+np.sqrt(E/2)*1j])


In [ ]:
X = np.array([])

for n in range(0,N_symbols,2):
    qpsk = np.sqrt(E/2)*(-(data[n]*2-1))+np.sqrt(E/2)*(-(data[n+1]*2-1))*1j
    X = np.append(X, qpsk)

print('QPSK signal:',X)
#노이즈 발생
Y = X + np.random.normal(0,sigma,len(X)) + np.random.normal(0,sigma,len(X))*1j

#노이즈 제거
#Y = X
print('received signal:',Y)

ML Detection

In [ ]:
S_to_Y = np.zeros(4)
Y_trans_bit = np.array([])

s1 = np.ravel(np.array([S_1.real, S_1.imag]), order='C')
s2 = np.ravel(np.array([S_2.real, S_2.imag]), order='C')
s3 = np.ravel(np.array([S_3.real, S_3.imag]), order='C')
s4 = np.ravel(np.array([S_4.real, S_4.imag]), order='C')


for n in range(len(Y)):
    Y_complex_to_point = np.array([Y[n].real, Y[n].imag])
    
    #S_1과의 거리
    S_to_Y[0]= np.linalg.norm(s1 - Y_complex_to_point)
    #S_2과의 거리
    S_to_Y[1]= np.linalg.norm(s2 - Y_complex_to_point)
    #S_3과의 거리
    S_to_Y[2]= np.linalg.norm(s3 - Y_complex_to_point)
    #S_4과의 거리
    S_to_Y[3]= np.linalg.norm(s4 - Y_complex_to_point)
    
    min = np.argmin(S_to_Y)
    if min == 0:
        Y_trans_bit = np.append(Y_trans_bit, np.array([0, 0]))
    elif min == 1:
        Y_trans_bit = np.append(Y_trans_bit, np.array([0, 1]))
    elif min == 2:
        Y_trans_bit = np.append(Y_trans_bit, np.array([1, 1]))
    elif min == 3:
        Y_trans_bit = np.append(Y_trans_bit, np.array([1, 0]))

print(Y_trans_bit)

X_detected = (Y_trans_bit > 0)
print(X_detected)

print('number of erros:', np.sum(data != X_detected))
print('error probability:',np.sum(data != X_detected)/data.shape)

LOOP for different SNR values

In [ ]:
SNR_dB = np.arange(0,11,1)
SNR_real = 10**(0.1*SNR_dB)
sigma = 1
E_array = SNR_real*sigma**2

print(SNR_dB)
print(SNR_real)
print(E_array)

In [ ]:
#N_symbols = 1000000
N_symbols = 20
P_e = np.zeros(SNR_dB.shape) # [0,0,0,0,0,0,0,0,0,0,0]

s1 = np.ravel(np.array([S_1.real, S_1.imag]), order='C')
s2 = np.ravel(np.array([S_2.real, S_2.imag]), order='C')
s3 = np.ravel(np.array([S_3.real, S_3.imag]), order='C')
s4 = np.ravel(np.array([S_4.real, S_4.imag]), order='C')

for n in range(len(E_array)):
    E = E_array[n]
    
    data = np.random.randint(0, 2, N_symbols)

    X = np.array([])

    for i in range(0,N_symbols,2):
        qpsk = np.sqrt(E/2)*(-(data[i]*2-1))+np.sqrt(E/2)*(-(data[i+1]*2-1))*1j
        X = np.append(X, qpsk)

    Y = X + np.random.normal(0,sigma,len(X)) + np.random.normal(0,sigma,len(X))*1j

    S_to_Y = np.zeros(4)
    Y_trans_bit = np.array([])

    for j in range(len(Y)):
        Y_complex_to_point = np.array([Y[j].real, Y[j].imag])
    
        #S_1과의 거리
        S_to_Y[0]= np.linalg.norm(s1 - Y_complex_to_point)
        #S_2과의 거리
        S_to_Y[1]= np.linalg.norm(s2 - Y_complex_to_point)
        #S_3과의 거리
        S_to_Y[2]= np.linalg.norm(s3 - Y_complex_to_point)
        #S_4과의 거리
        S_to_Y[3]= np.linalg.norm(s4 - Y_complex_to_point)
    
        min = np.argmin(S_to_Y)
        if min == 0:
            Y_trans_bit = np.append(Y_trans_bit, np.array([0, 0]))
        elif min == 1:
            Y_trans_bit = np.append(Y_trans_bit, np.array([0, 1]))
        elif min == 2:
            Y_trans_bit = np.append(Y_trans_bit, np.array([1, 1]))
        elif min == 3:
            Y_trans_bit = np.append(Y_trans_bit, np.array([1, 0]))

    P_e[n] = np.sum(data != (Y_trans_bit > 0))/data.shape

print(P_e)





In [ ]:
plt.semilogy(SNR_dB,P_e,linewidth=1, linestyle='--',color='r',marker='x',label='simulation')
plt.grid()
plt.legend()
plt.xlabel('SNR[dB]')
plt.ylabel('error prob.')
plt.show()


Your project: What is the error probability of QPSK for different SNR values? 